# IpyWidgets - The Basics   
This notebook illustrates basic ipywidget functionality  
For widgets used in combination with plotting and audio, see ipywidgets-mpl

----
Author: Dirk Van Compernolle   
Created: 24/11/2020   
Modifications:
> 17/03/2021: added more complex examples with multiple buttons and output widgets

Dependencies:   
> - ipywidgets
> - Ipython.display

Compatibility Issues:
> - generally robust across versions and platforms 
> - widget.Play() for animations does not work in Google Colab [Demo 6]
> - widget.Output() does not do text wrapping in Google Colab [Demo 8]

In [1]:
# [![Binder](http://mybinder.org/badge_logo.svg)](http://mybinder.org/v2/gh/binder-examples/conda_environment/master?filepath=index.ipynb)

----
### Elementary Widgets
----

#### 1. Slider Widget

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import IPython.display as ipd
import numpy as np

In [3]:
# widget1: integer slider
slider1 = widgets.IntSlider(max=20)
ipd.display(slider1)
def my_func(x):
    print(2*x)

IntSlider(value=0, max=20)

#### 2. Button Widget

##### Remarks on capturing output from ipywidgets and differences between Jupyter Notebook and Jupyter Lab
- Case 1:
    + A 'Button 1' is created and prints
    + In Notebook (and Colab) this is captured and redirected to the cell output
    + In Lab this is treated as an unspecified/orphaned output and redirected to the log
- Case 2:
    + An additional output widget is created and with the display function both displayed
    + The output is explicitly sent to the output widget
    + This works consistent across Notebook and Lab
    
##### Remarks on implicit argument
- The on_click() callback will be called with one argument, the clicked button widget instance
- The B2 example incorporates a state into the button as a counting memory is added

In [10]:
# simple button, but sending the output via output widget
button1 = widgets.Button(description="B1: Click me to Print")
def printMe(b):
    print("B1: Printing")
button1.on_click(printMe)
button1

Button(description='B1: Click me to Print', style=ButtonStyle())

In [30]:
# simple button, but sending the output via output widget
# state added: count number
# property (color) modified in function of state (even or odd count) 
button2 = widgets.Button(description="B2")
button2.count = 0
output2 = widgets.Output()
ipd.display(button2,output2)
def printMe(b):
    b.count += 1
    b.style.button_color = 'red' if b.count %2 else 'green'
    with output2:
        if b.count == 1: print(b.__dict__)
        print('\"',b.description,'\", count= ',b.count)
button2.on_click(printMe)

Button(description='B2', style=ButtonStyle())

Output()

In [24]:
# The BUTTON widget is specific

#### 3. A textbox  ... get's printed n times 

In [13]:
# widget 3: printing text
# note: this widget/print statement works in Jupyter Lab - no idea why (because there is no button involved or side-effect of the decorator ?)
@interact( text='', times=(0,5) )
def printIt(text,times):
    print( text * times )

interactive(children=(Text(value='', description='text'), IntSlider(value=2, description='times', max=5), Outp…

#### 4. A progress bar ...  counts your clicks

In [ ]:
progress_bar = widgets.IntProgress(
    value=0,
    min=0, max=10,step=1,
    description='Counting',
    bar_style='success',
    orientation='horizontal'
    )
btn_add = widgets.Button( description= 'add one ... ' )

def AddOne(b):
    progress_bar.value += 1
    if progress_bar.value == 10:
        progress_bar.bar_style=''
    elif progress_bar.value % 2 != 0:
        progress_bar.bar_style='danger'
    else:
        progress_bar.bar_style='success'
btn_add.on_click(AddOne)

widgets.HBox([btn_add, progress_bar])

#### 5. A dropdown menu ... doesn't do anything here

In [ ]:
rec_play = widgets.Dropdown(
    options={'REC','PLAY'},
    description='ACTION',
)
rec_play

#### 6. Linking widgets together with the Play widget

notes: Play widget not working in Colab

In [ ]:
# widget2: animation by Play
play2 = widgets.Play(
    interval=1000, # interval in msec between animation frame
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider2 = widgets.IntSlider()
widgets.jslink((play2, 'value'), (slider2, 'value'))
widgets.HBox([play2, slider2])

#### 7. Laying out an app Using Boxes and Packaging and app into a class 
- The outputs are displayed in controlled output widgets and can be cleared as wished
- Buttons and multiple outputs are neatly organized with Box, VBox and HBox
- Incorporating an app into a class has two main advantage:
    + It is straightforward to store information in the class
    + reusability of the same app

In [ ]:
# compose your app of a few controls and an output widget
# beautify by putting them all in boxes
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

b1 = widgets.Button(description='button 1')
b2 = widgets.Button(description='button 2')
b3 = widgets.Button(description='CLEAR')
b3.style.button_color = 'red'
output = widgets.Output()

def make_boxes():
    vbox1 = widgets.VBox([widgets.Label('Controls'), b1, b2])
    vbox2 = widgets.VBox([widgets.Label('CTRL-CLEAR'), b3 ])
    return vbox1, vbox2
 
vbox1, vbox2 = make_boxes()
vbox1.layout = make_box_layout()
vbox2.layout = make_box_layout()
output.layout = make_box_layout()
output.layout.width='200px'

def b1_clicked(b):
    with output:
        print("Button 1 clicked.")
b1.on_click(b1_clicked)
def b2_clicked(b):
    with output:
        print("Button 2 clicked.")
b2.on_click(b2_clicked)
def b3_clicked(b):
    with output:
        ipd.clear_output()
b3.on_click(b3_clicked) 

myapp = widgets.HBox([vbox1, vbox2,output])
myapp

In [31]:
# Packing a similar app into a class for reusability
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

class Button_Print(widgets.HBox):
    def __init__(self):
        super().__init__()
        
        b1 = widgets.Button(description='button 1')
        clear = widgets.Button(description='CLEAR')
        clear.style.button_color = 'red'
        output = widgets.Output()
        
        
        box1 = widgets.VBox([widgets.Label('Controls'), b1, clear])
        box1.layout = make_box_layout()
        output.layout = make_box_layout()
        output.layout.width='200px'
        
        # callback functions
        def b1_clicked(b):
            with output:
                print("Button 1 clicked.")
        b1.on_click(b1_clicked)

        def clear_clicked(b):
            with output:
                ipd.clear_output()

        clear.on_click(clear_clicked) 
        
        self.children = [box1,output]

myapp=Button_Print()
myapp

Button_Print(children=(VBox(children=(Label(value='Controls'), Button(description='button 1', style=ButtonStyl…

In [ ]:
myapp2=Button_Print()
myapp2

#### 8. Duplicating Similar Widgets with functools.partial()
- Similar components (eg. buttons) are created using functools.partial() method

In [ ]:
# now a similar app, with many buttons that have a similar behavior
# functools.partial() comes in handy to create all the similar callback routines
import functools

n_buttons = 5
buttons = []
box_layout = widgets.Layout(
        width='250px',
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
button_layout = widgets.Layout(
        width = '150px',
        height = '40px',
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

output1 = widgets.Output(layout=box_layout)
output2 = widgets.Output(layout=box_layout)
with output1:
    print("INFO:")
    print("All buttons do the same operation. However, they use the button number as a parameter")
    
def on_button_clicked(i,b):
    with output2:
        print("%d**2 = %d " % (i,i*i) )
        
for i in range(n_buttons):
    if i==0:
        button = widgets.Button(description='CLEAR',layout=button_layout)
        button.style.button_color = 'green'
        def clear_click(b):
            with output2:
                ipd.clear_output()                
        button.on_click(clear_click)
    else:
        button = widgets.Button(description='button '+str(i),layout=button_layout)
        button.on_click(functools.partial(on_button_clicked,i))        
    buttons.append(button)
    
button_box = widgets.VBox(buttons,layout=box_layout)
output_box = widgets.VBox([output1,output2])
myapp = widgets.HBox([button_box,output_box])
myapp

---
### GAME:  Beat the Bot (guessing game)
The bot learns from your previous clicking behavior   
Just try to beat him   
Have you found the optimum strategy

-----

In [ ]:
# some globals as needed
global user_history
user_history = [1,0]
target_score = 5

#buttons
btn0 = widgets.Button( description= '0' )
btn1 = widgets.Button( description= '1' )
btnS = widgets.Button( description= 'START' )

def click_zero(b):
    update_game(0)
btn0.on_click( click_zero )
def click_one(b):
    update_game(1)
btn1.on_click( click_one )
def click_start(b):
    update_game(0,start=True)
btnS.on_click( click_start )
btn0.disabled = True
btn1.disabled = True
btn0.button_style ='info'
btn1.button_style ='info'
btnS.button_style ='warning'
# score board
user_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='You',
    bar_style='success',
    orientation='horizontal'
    )
bot_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='Bot',
    bar_style='danger',
    orientation='horizontal'
    )
scoreboard= widgets.VBox((user_score,bot_score))
final_msg = widgets.HTML("<h1 style='color:green'> You win </h1>")
final_msg.layout.visibility = "hidden"
# layout
game = widgets.VBox( ( widgets.HBox((scoreboard,final_msg)),
                   widgets.HBox((btn0,btn1,btnS))
             ))

In [ ]:
#
# game rules
#
# note only user_history had to be defined as a global
# all widgets behave by nature as such

def update_game(user_choice,start=False):
    global user_history

    if( start ):
        user_history = [1,0]
        final_msg.value = "<h1 style='color:green'> You win </h1>"
        final_msg.layout.visibility = "hidden"
        user_score.value = 0
        bot_score.value = 0
        btn0.disabled = False
        btn1.disabled = False
        btnS.disabled = True
        btnS.button_style = ''
        return
    prob = sum(user_history)/len(user_history)
    comp_choice = np.random.binomial(1,prob,1)[0]
    user_history.append( user_choice )
    # print("sequence length: ",len(user_history)-2)
    if comp_choice == user_choice:
        bot_score.value += 1
    else:
        user_score.value += 1
    if user_score.value == target_score or bot_score.value == target_score:
        if bot_score.value == target_score:
            final_msg.value = "<h1 style='color:red'>Game Over</h1>"
        final_msg.layout.visibility = "visible"
        btn0.disabled = True
        btn1.disabled = True
        btnS.disabled = False
        btnS.button_style = 'warning'
    return

In [ ]:
game